# Practical ML: Scikit-learn
## Introducing Pandas and Scikit-learn
### Juan Antonio Cortés (jacortes@ugr.es)

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_predict, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.impute import KNNImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

# Pandas

Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language.

It is the perfect tool to explore, clean and process tabular data.

# Pandas series

Pandas tables (DataFrames from now on) are comprised of series (one-dimensional ndarray with axis labels). Below, an example of a series with a missing value.

In [ ]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

# Pandas DataFrame

DataFrames can be built from different data types. For example, we can build a DataFrame with a date column (it will be the index of the DataFrame in this example) and four numerical columns (the data).

In [ ]:
dates = pd.date_range('20201101', periods=6)
df = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))

In [ ]:
df

We can also build a DataFrame from a Python dictionary. Thanks to the flexibility of Pandas we can mix different data types inside the same DataFrame. For example: integers, time stamps, floats, text...

In [ ]:
df2 = pd.DataFrame({'A': 1.,
                    'B': pd.Timestamp('20201101'),
                    'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                    'D': np.array([3] * 4, dtype='int32'),
                    'E': pd.Categorical(["test", "train", "test", "train"]),
                    'F': 'foo'})

In [ ]:
df2

# Viewing data

Once we have our DataFrame we can use different approaches to view it. Pandas offers multiples ways of acomplish this task:

We can peek at the beginning of the DataFrame

In [ ]:
df.head()

Or the end:

In [ ]:
df.tail()

We can also list the name of the columns:

In [ ]:
df.columns

Transposing the data is a trivial task with Pandas:

In [ ]:
df.T

As well as sort the DataFrame. For example, we can sort it by column in descending order:

In [ ]:
df.sort_index(axis=1, ascending=False)

Or by column:

In [ ]:
df.sort_values(by='B')

Pandas also offers a quick statistic summary of the data:

In [ ]:
df.describe()

# Selection

Another usual task whilst working with data is the selection. Pandas offers a "pythonic" selection interface so we can slice our DataFrame in multiple ways. With Series, the syntax works exactly as with an ndarray, returning a slice of the values and the corresponding labels.

We can select by column:

In [ ]:
df['A']

In [ ]:
df[['A', 'C']]

By index:

In [ ]:
df[0:3]

Or by a combination of the previous method:

In [ ]:
df.iloc[1:3, :]

In [ ]:
df.iloc[:, 1:3]

# Boolean indexing

Pandas also supports boolean indexing. With this feature we can select data based on conditions, complex or simple.

For example, we can obtain the boolean mask that satisfies the following condition:

In [ ]:
mask = df > 0
mask

In [ ]:
df[mask]

We can also apply these conditions over a subset of the data. For example, we can get all the values that satisfies the next constraint:

In [ ]:
df[df['A'] > 0]

And of course the logical operators (and, or, exclusive-or) are supported:

In [ ]:
df[(df['A'] > 0) & (df['C'] > 0)]

# Missing data

Pandas can handle missing values without hassle. Given the Series introduced at the beginning of the notebook which contains one missing value:

In [ ]:
s

We can obtain which values are missing (NaN):

In [ ]:
pd.isna(s)

As we estated previously Pandas allows boolean indexing...

In [ ]:
s[pd.isna(s)]

At this point we are left with three choices: do nothing, delete the missing values or replace them with a value. The first one is not an issue and with Pandas none of the other two choices posed a threat.

We can easily delete them:

In [ ]:
s.dropna(how='any')

Or replace it with another value, in this case 5, but Pandas offers multiple options: previous/next valid value, mean or whichever function we pass to the `fillna` method.

In [ ]:
s.fillna(value=5)

# Plotting

Pandas offers a shortcut to `matplotlib` so we can easily plot our data with the call of a method. Lets build a random time series dataset and plot it.

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('11/1/2020', periods=1000))
ts = ts.cumsum()

The default plot type is the `line` plot, perfect for the data at hand.

In [ ]:
ts.plot()

We can also plot multivariate data and Pandas will handle the labels, legends, colours...

In [ ]:
df.plot()

Or change the plot type:

In [ ]:
df.plot(kind='bar')

# Data reading

And if you are wondering Pandas can also read (and write) from multiple file types: CSV, Excel, HDF5, Parquet... Lets load the SDSS (Sloan Digital Sky Survey) dataset from https://www.sdss.org/.

In [ ]:
df =  pd.read_csv('data/sdss.csv')
_ = df.pop('objid')

But this is not all about Pandas. Pandas is a huge library which offers endless resources to handle our data. Do not forget to visit the official documention to take a glance of all that Pandas has to offer: https://pandas.pydata.org/docs/reference/index.html#api

# EDA (Exploratory Data Analysis)

We can start by taking a peek at the head of the data to get a notion of the data at hand. The data consists of 10000 observations of space taken by the SDSS (Sloan Digital Sky Survey). Every observation is described by 17 feature columns and 1 class column which identifies it to be either a star, galaxy or quasar.

Data from: https://www.sdss.org/

The Sloan Digital Sky Survey has created the most detailed three-dimensional maps of the Universe ever made, with deep multi-color images of one third of the sky, and spectra for more than three million astronomical objects. Learn and explore all phases and surveys—past, present, and future—of the SDSS.


In [ ]:
df.head()

The columns description is as follows:
* `ra`: right ascension.
* `dec`: declination.
* `u`: ultraviolet.
* `g`: green.
* `r`: red.
* `i`: near infrared.
* `z`: infrared.
* `run`: run number.
* `rerun`: rerun number.
* `camcol`: camera column.
* `field`: field number.
* `specobjid`: object Identifier.
* `class`: object class (galaxy, star or quasar).
* `redshift`: final redshift.
* `plate`: plate number.
* `mjd`: MJD of observation.
* `fiberid`: fiber ID.


After knowing a bit about the data we can check for missing values:

In [ ]:
pd.isna(df) #.any()

Another basic task during the EDA phase is to check the class distribution. We can easily plot the histogram with Pandas:

In [ ]:
df['class'].value_counts().plot(kind='bar')

The data is clearly imbalanced towards the GALAXY and STAR classes.

In [ ]:
df['class'].value_counts()

---

# Scikit-learn

Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

# Workflow

The next figure shows the basic workflow while approaching a Machine Learning problem. We can distinguish two tasks, separate at first but intertwined in the last stages:

* Optimization of the machine learning model (left) which will make the prediction using the data.
* Construction of the dataset (accomplished during the EDA phase) and partitioning in training and test data (right).

![](data/grid_search_workflow.png)
From: https://scikit-learn.org/stable/modules/cross_validation.html

# Data split

We must first split the data in two sets: `X` and `y`. This names are a common convention. `X` for the variables and `y` for the target (class).

In [ ]:
y = df.pop('class')
X = df

The next step is to split the data, once more time, into train and test partitions. Scikit-learn can easily do that for us.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=24112020)

We get the train partition (80% of our data) and the test partition (20% of our data) randomly selected from all the data and stratified (both partitions has the same class distribution) for better results.

# Cross Validation

Next, we train and optimize the desired model via the cross validation strategy. The next figure shows a graphical representation of this strategy:

![](data/grid_search_cross_validation.png)
From: https://scikit-learn.org/stable/modules/cross_validation.html

# kNN

![](data/knn_example.png)
From: Cheng, Debo, et al. "kNN algorithm with data-driven k value." International Conference on Advanced Data Mining and Applications. Springer, Cham, 2014.

First of all, we need to define the parameters which will be tested. In this case, the number of neighbors (`k`) that the algorithm will use.

In [ ]:
parameters = {'n_neighbors': [x for x in range (3, 16)]}

Next, we instantiate our classifier.

In [ ]:
clf = KNeighborsClassifier()

Now we can search for the best `k`. Sklearn offers the `GridSearchCV` method to acomplish this. This method will test all the combination of parameters via cross validation and will return the best parameters obtained during this traning phase.

In [ ]:
clf = GridSearchCV(clf, parameters, refit=True, cv=StratifiedKFold(), n_jobs=4)

In [ ]:
clf.fit(X_train, y_train)

In this example, we get `k=15`.

In [ ]:
clf.best_estimator_

In [ ]:
clf = clf.best_estimator_

After the training step we need to check how the model performs against data that has not been "seen" by the model (the test data).

In [ ]:
y_pred = clf.predict(X_test)

After we get the predictions we compare the to the ground-truth to check the model's performance. Once again, sklearn has a classfication report with all the common metrics used in this type of problems. 

In [ ]:
print(classification_report(y_test, y_pred))

# Decision Tree

But what if we use a more complex (yet simple) model? The next figure shows a simple decision tree.

![](data/dt_example.png)
From: https://towardsai.net/p/programming/decision-trees-explained-with-a-practical-example-fe47872d3b53

In [ ]:
parameters = {'criterion':('gini', 'entropy'), 
              'max_depth':[5, 10, 15, None]}

In [ ]:
clf = tree.DecisionTreeClassifier(random_state=24112020)

In [ ]:
clf = GridSearchCV(clf, parameters, refit=True, cv=StratifiedKFold(), n_jobs=4)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.best_estimator_

In [ ]:
clf = clf.best_estimator_

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

One of the perks of this model is the interpretability of it. Thanks to sklearn, we can plot the final decision tree:

In [ ]:
plt.figure(figsize=(40,20))
tree.plot_tree(clf, feature_names=X.columns, class_names=['GALAXY', 'QSAR', 'STAR']);

# Random Forest

Random forests are an ensemble learning method for classification, regression and other tasks that operate by constructing a multitude of decision trees.

![](data/rf_example.png)
From: https://rpubs.com/Avalos42/randomforest

In [ ]:
parameters = {'criterion': ['gini', 'entropy'],
              'n_estimators': [5, 10, 15, 20],
              'max_depth': [5, 10, 15, None]}

In [ ]:
clf = RandomForestClassifier(random_state=24112020)


In [ ]:
clf = GridSearchCV(clf, parameters, refit=True, cv=StratifiedKFold(), n_jobs=4)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.best_estimator_

In [ ]:
clf = clf.best_estimator_

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

This model offers an useful trait: the feature importance. The higher the value the more important is the feature (variable) to the model so we can discard useless variables and improve the performance of the model. This is called "feature selection" and is one of the most powerful preprocessing techniques.

In [ ]:
feature_importance = clf.feature_importances_
feature_importance

In [ ]:
fs_df = pd.DataFrame([X.columns, feature_importance]).T
fs_df.columns = ['Feature', 'Importance']
fs_df.sort_values(by='Importance', ascending=False)

As we can see the `redshift` is the more important feature by a huge margin. Lets see what happens when we train another model with only this feature.

# Support Vector Machine (SVM)

A support-vector machine (SVM) constructs a hyperplane or set of hyperplanes in a high- or infinite-dimensional space, which can be used for classification, regression, or other tasks like outliers detection.

![](data/svm_example.png)
From: Ulas, Cagdas. (2013). INCORPORATION OF A LANGUAGE MODEL INTO A BRAIN COMPUTER INTERFACE BASED SPELLER. 10.13140/2.1.2017.6326. 

In [ ]:
parameters = {'svc__kernel': ['rbf', 'linear', 'poly']}

The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters. For this, it enables setting parameters of the various steps using their names and the parameter name.

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(random_state=24112020))

In [ ]:
clf = GridSearchCV(clf, parameters, refit=True, cv=StratifiedKFold(), n_jobs=4)

In [ ]:
clf.fit(X_train['redshift'].values.reshape(-1, 1), y_train)

In [ ]:
clf.best_estimator_

In [ ]:
clf = clf.best_estimator_

In [ ]:
y_pred = clf.predict(X_test['redshift'].values.reshape(-1, 1))

As we can see we obtain "nearly perfect" results using only one feature. This is a huge improvement in terms of traning speed, prediction speed and model complexity (less complex models are often prefered)

In [ ]:
print(classification_report(y_test, y_pred))

# Clustering (K-Means)

Lastly, lets try an unsupervised learning (a type of machine learning that looks for previously undetected patterns in a data set with no pre-existing labels and with a minimum of human supervision) model for clustering: K-Means

## Choosing the best number of cluster: the elbow method

In cluster analysis, the elbow method is a heuristic used in determining the number of clusters in a data set. The method consists of plotting the explained variation as a function of the number of clusters, and picking the elbow of the curve as the number of clusters to use.

In [ ]:
distorsions = []
labels = []
centroids = []
X = X[['ra', 'dec', 'redshift']]
X_test = X_test[['ra', 'dec', 'redshift']]
for k in range(2, 10):
    clf = KMeans(n_clusters=k, random_state=24112020)
    clf.fit(X)
    labels.append(clf.predict(X_test))
    distorsions.append(clf.inertia_)
    centroids.append(clf.cluster_centers_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 10), distorsions)
plt.grid(True)
plt.title('Elbow curve')

In [ ]:
#%matplotlib notebook

In [ ]:
plt.figure('K-Means on SDSS Dataset', figsize=(10,10))
ax = plt.axes(projection='3d')
ax.scatter(centroids[1][0][0], centroids[1][0][1], centroids[1][0][2], c='r', s=100)
ax.scatter(centroids[1][1][0], centroids[1][1][1], centroids[1][1][2], c='r', s=100)
ax.scatter(centroids[1][2][0], centroids[1][2][1], centroids[1][2][2], c='r', s=100)
scatter = ax.scatter(X_test['ra'], X_test['dec'], X_test['redshift'], c=labels[1])
plt.legend(handles=scatter.legend_elements()[0], labels=['GALAXY', 'QSO', 'STAR'])